In [1]:
import torch
from map_action_tranform import get_transform
from map_action_data_loader import map_action_test_data_loader, map_action_test_dataset 
from m_a_detection_model import map_action_instance_segmentation_model
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, accuracy_score, average_precision_score
from sklearn.metrics import precision_recall_curve
import numpy as np
import matplotlib.pyplot as plt


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [2]:
# Function to calculate IoU
def calculate_iou(pred_boxes, true_boxes):
    x1 = torch.max(pred_boxes[:, 0], true_boxes[:, 0])
    y1 = torch.max(pred_boxes[:, 1], true_boxes[:, 1])
    x2 = torch.min(pred_boxes[:, 2], true_boxes[:, 2])
    y2 = torch.min(pred_boxes[:, 3], true_boxes[:, 3])

    intersection = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)
    area_pred = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])
    area_true = (true_boxes[:, 2] - true_boxes[:, 0]) * (true_boxes[:, 3] - true_boxes[:, 1])
    union = area_pred + area_true - intersection

    iou = intersection / union
    return iou

In [27]:
import torch
import numpy as np
from sklearn.metrics import average_precision_score, accuracy_score, precision_score, precision_recall_curve
import matplotlib.pyplot as plt

def test_model(model, test_loader, device):
    model.eval()

    all_pred_boxes = []
    all_true_boxes = []

    for images, targets in test_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        with torch.no_grad():
            outputs = model(images)

        for output, target in zip(outputs, targets):
            pred_boxes = output['boxes'].cpu()
            true_boxes = target['boxes'].cpu()
            all_pred_boxes.append(pred_boxes)
            all_true_boxes.append(true_boxes)

    all_pred_boxes = torch.cat(all_pred_boxes, dim=0)
    all_true_boxes = torch.cat(all_true_boxes, dim=0)

    ious = calculate_iou(all_pred_boxes, all_true_boxes)
    average_precision = average_precision_score(np.ones(len(ious)), ious)

    # Threshold for considering a detection correct
    detection_threshold = 0.5
    pred_labels = (ious > detection_threshold).float()

    true_labels = torch.ones(len(all_true_boxes), dtype=torch.float32)

    # Convert tensors to numpy arrays
    true_labels_np = true_labels.numpy().astype(int)
    ious_np = ious.numpy().astype(float)

    # Ensure that both arrays have the same length
    min_length = min(len(true_labels_np), len(ious_np))
    true_labels_np = true_labels_np[:min_length]
    ious_np = ious_np[:min_length]

    # Ensure that there are at least two unique values in true_labels_np to avoid the ValueError
    unique_labels = np.unique(true_labels_np)
    if len(unique_labels) < 2:
        print("Skipping precision-recall curve plot due to insufficient unique labels.")
    else:
        precision, recall, _ = precision_recall_curve(true_labels_np, ious_np)
    
        accuracy = accuracy_score(true_labels_np, pred_labels.numpy().astype(int))
        precision = precision_score(true_labels_np, pred_labels.numpy().astype(int))

        print(f'Mean Average Precision (mAP): {average_precision:.4f}')
        print(f'Accuracy: {accuracy:.4f}')
        print(f'Precision: {precision:.4f}')

        # Plotting precision-recall curve
        plt.plot(recall, precision, marker='.')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.show()


In [28]:
root_dir_test = '/home/mapaction/Documents/Exp-data/Coco/images/'
annotation_file_test = '/home/mapaction/Documents/Exp-data/Coco/result.json'

# Load the pre-trained model
model_path = '/home/mapaction/mapaction_env/Map-Action-Model/model/MAISM1.pth'
model = map_action_instance_segmentation_model(2)  # Initialize your model architecture
model.load_state_dict(torch.load(model_path))
model.eval()

# Create the testing dataset
test_transform = get_transform
test_dataset = map_action_test_dataset
  # Replace with your class names

# Create a data loader for testing
test_loader = map_action_test_data_loader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

test_model(model, test_loader, device)


cuda
Skipping precision-recall curve plot due to insufficient unique labels.
